In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import pickle
from sklearn.model_selection import train_test_split


In [2]:
data=pd.read_csv('output/integrators_log.csv')
data.drop('Responsive',axis=1,inplace=True)

In [3]:
traces = [group.values.tolist() for _, group in data.groupby('Case ID')]

In [4]:
traces[0]

[['Exam',
  0,
  9.276767459125256,
  145.31414076463105,
  9.113142839882707,
  nan,
  1.0749042576137682],
 ['hemo_int', 0, nan, nan, nan, 3.923232540874743, 47.56877744321062],
 ['Exam',
  0,
  13.2,
  145.31414076463105,
  9.113142839882707,
  nan,
  0.6759146998838428],
 ['pot_inib', 0, nan, nan, nan, 1.8065714199413545, 47.84485865496066],
 ['Exam',
  0,
  13.2,
  145.31414076463105,
  7.306571419941354,
  nan,
  0.5309334260417937],
 ['pot_inib', 0, nan, nan, nan, 0.4516428549853386, 48.98919362782402],
 ['Exam',
  0,
  13.2,
  145.31414076463105,
  6.854928564956015,
  nan,
  0.537744134949508],
 ['pot_inib', 0, nan, nan, nan, 0.2258214274926692, 49.6734409666479],
 ['Exam',
  0,
  13.2,
  145.31414076463105,
  6.629107137463346,
  nan,
  0.3176814987563396],
 ['pot_inib', 0, nan, nan, nan, 0.1411383921829182, 47.35307538810872],
 ['Exam',
  0,
  13.2,
  145.31414076463105,
  6.487968745280428,
  nan,
  1.464021413194395],
 ['pot_inib', 0, nan, nan, nan, 0.0987968745280428, 47.

In [4]:
exam_therapy=[]

for trace in traces:
    case=[]
    for i in range(0,len(trace)-1,2):
        moment=trace[i][2:5]
        moment.append(trace[i+1][0])
        moment.append(trace[i+1][5])
        case.append(moment)
    exam_therapy.append(case)

In [5]:
train,test=train_test_split(exam_therapy,test_size=0.2)

In [6]:
activities = ['hemo_int', 'hemo_inib',
       'sodium_int', 'sodium_inib', 'pot_int',
       'pot_inib', 'recovered',
       'Non responsive']

In [7]:
n_activities = len(activities)
map_activities = dict(zip(activities, [i+3 for i in range(n_activities)]))


def zeros(n):
    return [0 for _ in range(n)]


def hot_encode_activity(activity):
    encode=activity[:3]
    encode.extend(zeros(n_activities))
    encode[map_activities[activity[3]]] = 1
    encode.append(activity[-1])
    return encode

def hot_encode_trace(trace):
    return [hot_encode_activity(activity) for activity in trace]

def trace_to_tensor(trace):
    return torch.tensor(hot_encode_trace(trace), dtype= torch.float32 )

In [8]:
transformedTraces_train=[trace_to_tensor(trace) for trace in train]

transformedTraces_test=[trace_to_tensor(trace) for trace in test]

In [9]:
for trace in transformedTraces_train:
  nan_mask=torch.isnan(trace)
  trace[nan_mask]=0


for trace in transformedTraces_test:
  nan_mask=torch.isnan(trace)
  trace[nan_mask]=0


tensor([[1.6840e+01, 1.3627e+02, 8.0496e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 1.2748e+00],
        [1.6840e+01, 1.3627e+02, 6.7748e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 3.1870e-01],
        [1.6840e+01, 1.3627e+02, 6.4561e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 1.5935e-01],
        [1.6840e+01, 1.3627e+02, 6.2968e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 9.9594e-02],
        [1.6840e+01, 1.3627e+02, 6.1972e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 6.9716e-02],
        [1.6840e+01, 1.3627e+02, 6.1274e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 5.2287e-02],
        [1.6840e+01, 1